In [1]:
import sqlite3, os
import pandas as pd

In [2]:
conn = sqlite3.connect(r"logistics.db")
cursor = conn.cursor()

CREATE TABLES

In [ ]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS clientes (
    cif_empresa VARCHAR(50) NOT NULL PRIMARY KEY,
    nombre_cliente VARCHAR(100),
    direccion VARCHAR(100),
    telefono_cliente VARCHAR(20),
    email_cliente VARCHAR(100)
    )
""")



In [ ]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS pedidos (
    id_pedido INTEGER NOT NULL,
    cif_empresa VARCHAR(50) NOT NULL,
    id_producto INTEGER NOT NULL,
    destino VARCHAR(100),
    provincia VARCHAR(50),
    cantidad INT,
    fecha_pedido DATE,
    fecha_caducidad DATE,
    fecha_entrega_estimada DATE,
    estado VARCHAR(50),
    fecha_entregado DATETIME,
    PRIMARY KEY (id_pedido, cif_empresa, id_producto),
    FOREIGN KEY (provincia) REFERENCES destinos (provincia),
    FOREIGN KEY (cif_empresa) REFERENCES clientes (cif_empresa),
    FOREIGN KEY (id_producto) REFERENCES productos (id_producto)
    )
""")



In [ ]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS productos (
    id_producto INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
    nombre_producto VARCHAR(100),
    precio INT,
    tiempo_fabricacion INT,
    caducidad INT,
    categoria VARCHAR(50)
    )
""")



In [ ]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS destinos (
    provincia VARCHAR(100) NOT NULL,
    latitud FLOAT NOT NULL,
    longitud FLOAT NOT NULL,
    PRIMARY KEY (provincia,latitud,longitud),
    FOREIGN KEY (provincia) REFERENCES clientes (direccion)
    )
""")

In [ ]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS camiones (
    id_camion INTEGER NOT NULL,
    id_pedido INTEGER NOT NULL,
    id_producto INTEGER NOT NULL,
    cantidad_pedido INTEGER NOT NULL,
    destino VARCHAR(50) NOT NULL,
    ruta VARCHAR(50),
    PRIMARY KEY (id_camion,id_pedido,id_producto,cantidad_pedido),
    FOREIGN KEY (id_pedido) REFERENCES pedidos(id_pedido),
    FOREIGN KEY (id_producto) REFERENCES productos(id_producto),
    FOREIGN KEY (destino) REFERENCES destinos(provincia)
    )
""")

INSERT VALUES

In [11]:
path = r"..\data"

for csv in os.listdir(path):
    df = pd.read_csv(os.path.join(path, csv), sep=",") 
    df.to_sql(csv.split(".")[0], conn, if_exists='replace', index=False)


CREAR PEDIDOS RANDOM

In [6]:
import random, csv
from datetime import datetime, timedelta

with open("200_pedidos.csv", mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    writer.writerow([
        "id_pedido", "cif_empresa", "id_producto", "nombre_producto", "destino", "provincia", "cantidad", 
        "fecha_pedido", "fecha_caducidad", "fecha_entrega_estimada", "estado", "fecha_entregado"
    ])

    # Simulación de inserciones en pedidos
    for i in range(200):
        # Generar un ID de pedido único
        id_pedido = random.randint(1000000, 9999999)
        
        # Seleccionar cliente aleatorio
        client_id = cursor.execute(
            "SELECT cif_empresa FROM clientes ORDER BY RANDOM() LIMIT 1"
        ).fetchone()
        if client_id:  # Asegurarse de que no sea None
            client_id = client_id[0]
        
        # Obtener el destino basado en el cliente
        destino = cursor.execute(
            """
            SELECT provincia, latitud, longitud 
            FROM destinos 
            WHERE provincia = (SELECT direccion FROM clientes WHERE cif_empresa = ?)
            """, (client_id,)
        ).fetchone()
        
        if destino:
            provincia, latitud, longitud = destino

        # Seleccionar un producto aleatorio
        producto = cursor.execute(
            "SELECT * FROM productos ORDER BY RANDOM() LIMIT 1"
        ).fetchone()
        if producto:
            id_producto = producto[0]
            nombre_producto = producto[1]
            tiempo_fabricacion = producto[3]
            caducidad = producto[4]
        
        # Generar datos del pedido
        cantidad = random.randint(550, 990)
        fecha_pedido = datetime.today()
        fecha_caducidad = fecha_pedido + timedelta(days=caducidad)
        fecha_entrega_estimada = fecha_caducidad - timedelta(days=3)
        estado = "En fabricación"
        fecha_entregado = None

        writer.writerow(
            [
                id_pedido, client_id, id_producto, nombre_producto, f"{latitud}, {longitud}", provincia, cantidad, 
                fecha_pedido.strftime("%Y-%m-%d"), 
                fecha_caducidad.strftime("%Y-%m-%d"), 
                fecha_entrega_estimada.strftime("%Y-%m-%d"), 
                estado, fecha_entregado
            ]
        )

        # # Insertar en la tabla de pedidos
        # cursor.execute("""
        #     INSERT INTO pedidos (
        #         id_pedido, cif_empresa, id_producto, nombre_producto, destino, provincia, cantidad, 
        #         fecha_pedido, fecha_caducidad, fecha_entrega_estimada, 
        #         estado, fecha_entregado
        #     ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        # """, (
        #     id_pedido, client_id, id_producto,
        #     nombre_producto,f"{latitud}, {longitud}", provincia, cantidad, 
        #     fecha_pedido.strftime("%Y-%m-%d"), 
        #     fecha_caducidad.strftime("%Y-%m-%d"), 
        #     fecha_entrega_estimada.strftime("%Y-%m-%d"), 
        #     estado, fecha_entregado
        # ))

    # Confirmar cambios
    conn.commit()

_______________________________________________________________________________________________________________________________________________________________________

INTERACTUAR AMB BDD

In [ ]:
import os, sqlite3

def connectar_BDD():
    ruta_bd = f"{os.getcwd()}/logistics.db"
    try:
        connexio = sqlite3.connect(ruta_bd)
        cursor = connexio.cursor()
    except Exception as e:
        print(e)
    return connexio, cursor

def guardar_tancar_BDD(connexio):
    try:
        connexio.commit()
        connexio.close()
        return True
    except Exception as e:
        print(e)
        return False

connexio, cursor = connectar_BDD()
query = str(input("Logistic>> "))
while query != "q":
    if query == "cls":
        os.system("cls")
    else:
        try:
            cursor.execute(query)
            for line in cursor.fetchall():
                print(' | '.join(str(value) for value in line))
        except Exception as e:
            print(e)
    query = str(input("Logistic>> "))
guardar_tancar_BDD(connexio)
